In [5]:
import pandas as pd
import numpy as np

import cv2
import matplotlib.image as mpimg
import matplotlib.pylab as plt

ModuleNotFoundError: No module named 'cv2'

In [2]:
import sys
!{sys.executable} -m pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
    --------------------------------------- 0.5/38.6 MB 17.2 MB/s eta 0:00:03
   --- ------------------------------------ 3.0/38.6 MB 37.8 MB/s eta 0:00:01
   -------- ------------------------------- 8.0/38.6 MB 63.7 MB/s eta 0:00:01
   -------------- ------------------------- 13.7/38.6 MB 108.8 MB/s eta 0:00:01
   -------------------- ------------------- 19.4/38.6 MB 108.8 MB/s eta 0:00:01
   ------------------------- -------------- 25.0/38.6 MB 131.2 MB/s eta 0:00:01
   ------------------------------- -------- 30.8/38.6 MB 108.8 MB/s eta 0:00:01
   ------------------------------------- -- 36.4/38.6 MB 108.8 MB/s eta 0:00:

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/archive/annotations.csv')

In [ ]:
df

,file_name,width,height,x1,y1,x2,y2,category
0,000_0001.png,134,128,19,7,120,117,0
1,000_0002.png,165,151,23,12,149,138,0
2,000_0003.png,128,122,22,14,116,105,0
3,000_0010.png,80,73,14,8,67,63,0
4,000_0011.png,186,174,36,15,155,157,0
...,...,...,...,...,...,...,...,...
6159,056_1_0018_1_j.png,122,94,25,20,80,79,56
6160,056_1_0019_1_j.png,224,207,39,39,188,178,56
6161,056_1_0020_1_j.png,128,115,32,30,89,79,56
6162,057_1_0001_1_j.png,100,95,21,22,74,75,57


In [6]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import os
import shutil

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
data_dir = '/content/drive/MyDrive/archive/images'

In [ ]:
kategorije = set(df['category'])

In [ ]:
os.mkdir('data')

In [ ]:
for label in kategorije:
  directory = '{0:03}'.format(label)
  parent_dir = 'data'
  path = os.path.join(parent_dir, directory)
  os.mkdir(path)

In [ ]:
for i in range(df.size):
  try:
    shutil.copy(data_dir + '/' + df['file_name'][i], '/content/data/' + '{0:03}'.format(df['category'][i])+'/'+df['file_name'][i])
  except:
    print(i)

In [ ]:
data_dir = '/content/data'

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/content/data')

Found 5998 files belonging to 58 classes.


In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[0].shape

(32, 256, 256, 3)

In [ ]:
scaled = batch[0] / 255

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()[0].max()

1.0

In [ ]:
train_size = int(len(data)* 0.7)
validation_size = int(len(data)* 0.2) + 1
test_size = int(len(data)* 0.1) + 1

In [ ]:
len(data) - train_size - test_size - validation_size

0

In [ ]:
train = data.take(train_size)
validation = data.skip(train_size).take(validation_size)
test = data.skip(train_size + validation_size).take(test_size)

In [ ]:
len(validation)

38

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3, 3), activation='relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(os.listdir('/content/data')), activation='softmax'))

In [ ]:
model.compile('adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 254, 254, 128)     3584      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 127, 127, 128)     0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 125, 125, 64)      73792     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 30, 30, 128)      

In [ ]:
logdir = 'logs'
os.mkdir(logdir)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [ ]:
history = model.fit(train, epochs = 20, validation_data = validation, callbacks = [tensorboard_callback])

Epoch 1/20
131/131 [==============================] - 1920s 15s/step - loss: 2.9765 - accuracy: 0.2257 - val_loss: 1.9873 - val_accuracy: 0.4507
Epoch 2/20
124/131 [===========================>..] - ETA: 1:33 - loss: 1.7765 - accuracy: 0.4740

In [ ]:
history.history

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, Accuracy

In [ ]:
precision = Precision()
recall = Recall()
accuracy = Accuracy()

In [ ]:
for batch in test.as_numpy_iterator():
  X, y = batch
  yhat = tf.argmax(model.predict(X).T)
  precision.update_state(y, yhat)
  recall.update_state(y, yhat)
  accuracy.update_state(y, yhat)

1/1 [==============================] - 0s 32ms/step


In [ ]:
print(accuracy.result().numpy())
print(recall.result().numpy())
print(precision.result().numpy())

0.9932203
1.0
1.0


In [ ]:
from PIL import Image

In [ ]:
proba = cv2.imread('/content/proba.png')
proba_resized = cv2.resize(proba, (256, 256))

In [ ]:
proba_resized.shape

(256, 256, 3)

In [ ]:
for i in (model.predict(proba_resized.reshape(1, 256, 256, 3)/255).T):
  print(round(i[0]))

1/1 [==============================] - 0s 18ms/step
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
print(type(proba))

<class 'numpy.ndarray'>


In [ ]:
proba_resized = cv2.resize(proba, (256, 256, 3))